<h1> Sistemas Inteligentes </h1>
<h1> Práctica de Aprendizaje Supervisado </h1>

En esta práctica vamos a ver cómo se hace el entrenamiento de un método de aprendizaje supervisado en la librería scikit learn para python.

Las tareas a realizar son:
-  Cargar el conjunto de datos
-  Preparar los conjuntos de entrenamiento y prueba
-  Probar el modelo construido con un conjunto de prueba
-  Aplicar métricas de desempeño para evaluar el desempeño del modelo

Mayo de 2020 <br/>
Autor: G. Alvarez

In [1]:
# Documentación
# X_Train : los datos con los que se va entrenar el modelo
# y_train : Las respuestas de ese conjunto de datos (X_Train)
# X_test : son un conjunto de datos diferente para probar el modelo
# y_test : es la respuesta de ese conjunto de datos diferente

In [22]:
# GLOBAR VARIABLES
SVM = [0,0,0]
ARBOL = [0,0,0]
REGRESION = [0,0,0]

In [23]:
# Carga de librerías y lectura del archivo que contiene los datos

import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

url="https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
data = pd.read_csv(url, header=None, na_values=" ?")

#Ponemos nombre a las columnas (Esta información se toma del archivo original adult.names que está en el 
#repositorio junto con el archivo de datos)
data.columns = ['Age', 'Workclass', 'Fnlwgt', 'Education', 'Education-num', 'Marital-status', 'Occupation',
              'Relationship', 'Race', 'Sex', 'Capital-gain', 'Capital-loss', 'Hpw', 'Country', 'C']
#data

In [24]:
#Cuál es el número de registros?
#Cuál es el número de atributos?

shape = data.shape
shape

(32561, 15)

In [25]:
#Cuantos registros hay por cada clase? es decir, por cada valor del atributo de salida?

print(data['C'].value_counts())

 <=50K    24720
 >50K      7841
Name: C, dtype: int64


Las actividades siguientes corresponden al preprocesamiento de los datos para poderlos utilizar en el entrenamiento. Esta es una etapa importante y necesaria, a continuación se muestra la forma cómo se hace, aunque el propósito de la práctica tiene que ver más con la realización del entrenamiento, por lo que no se explicará en detalle.

In [26]:
# Eliminar los registros que tienen más de 2 datos faltantes
data = data.dropna(axis = 0, thresh = 13)
print(data.shape)

# Reemplazar los datos faltantes por la moda en los atributos Workclass, Occupation, Country
data.Workclass.fillna(data.Workclass.mode()[0], inplace=True)
data.Occupation.fillna(data.Workclass.mode()[0], inplace=True)
data.Country.fillna(data.Workclass.mode()[0], inplace=True)

# Convertir los atributos categóricos a escala numérica
# Esto modifica los valores de todas las columnas, incluso las numéricas
le = preprocessing.LabelEncoder()
data = data.apply(le.fit_transform) 

# Balanceo entre clases
g = data.groupby('C')
dataBal = pd.DataFrame(g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True)))
dataBal.shape

(32534, 15)


(15674, 15)

In [27]:
# Separacion de los datos en conjuntos de entrenamiento, validacion y prueba. Se trabaja sobre el conjunto balanceado
# Cuando finalice la depuración recordar quitar el parámetro random_state.
X_train, X_test, y_train, y_test = train_test_split(dataBal.drop(['C'],axis=1), dataBal['C'], test_size=0.4, random_state=42)

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)


X_train: (9404, 14)
X_test: (6270, 14)
y_train: (9404,)
y_test: (6270,)


In [34]:
# Estimación de parámetros
# En este código se ilustra el uso de la función que trae la librería sklearn para estimar paramentros,
# la cual no necesita la extracción explícita del conjunto de validación pues hace la estimación usando
# validación cruzada sobre el conjunto de entrenamiento.

from sklearn.svm import SVC
#from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def SVM_FUN(sizeTest, X_train, X_test, y_train, y_test):
    # Se crea el clasificador base
    clf = SVC()

    # Se definen los valores a explorar para cada parametro a estimar
    parameter_space = [{'kernel': ['rbf', 'sigmoid'], 
    #parameter_space = [{'kernel': ['rbf'], 
                         'gamma': [1e-3, 1e-4],
                         'C': [1, 10, 100]
                        }]

    # Se realiza la estimacion de parametros, en clf queda el modelo construido con los mejores
    # parametros encontrados y reentrenado con el conjunto de datos completo (refit)
    clf = GridSearchCV(clf, parameter_space, n_jobs=-1, cv=3, refit=True)
    clf.fit(X_train,y_train)

    # Se identifican los parametros que producen el mejor modelo
    print("Mejores parametros:")
    print(clf.best_params_)

    # Se hace la prediccion sobre los datos de prueba
    pred = clf.predict(X_test)

    # Se calculan metricas a partir de los datos de prueba
    mat = confusion_matrix(y_test, pred)
    print(mat)
    
    # Metricas
    pre = precision_score(y_test,pred)
    rec = recall_score(y_test,pred)
    f1Score = f1_score(y_test,pred)
    
    # Calculo en el vector de las metricas acumulando
    SVM[0] = SVM[0] + pre       # Presición
    SVM[1] = SVM[1] + rec       # Recall
    SVM[2] = SVM[2] + f1Score   # F1 Score
    
    print("Precision: ", pre)
    print("Recall:    ", rec)
    print("F1score:   ",f1Score)
    print("Reporte",classification_report(y_test, pred))

In [29]:
#Punto 2
# Estimación de parámetros
# En este código se ilustra el uso de la función que trae la librería sklearn para estimar paramentros,
# la cual no necesita la extracción explícita del conjunto de validación pues hace la estimación usando
# validación cruzada sobre el conjunto de entrenamiento.

from sklearn.svm import SVC
#from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def ArbolDesicion(sizeTest, profundidadArbol, X_train, X_test, y_train, y_test):
    # Clasificador de arbol de desiciones
    arbol = DecisionTreeClassifier()
    
    # Entrenamiento del modelo
    arbol.fit(X_train, y_train)
    
    # Calculo de score en el modelo de test
    scoTest = arbol.score(X_test, y_test)
    
    # Calculo de score en el modelo de entrenamiento
    scoTrain = arbol.score(X_train, y_train)
    
    # Predicción
    pred = arbol.predict(X_test)
    
    # Metricas
    pre = precision_score(y_test,pred)
    rec = recall_score(y_test,pred)
    f1Score = f1_score(y_test,pred)
    
    # Calculo en el vector de las metricas acumulando
    ARBOL[0] = ARBOL[0] + pre       # Presición
    ARBOL[1] = ARBOL[1] + rec       # Recall
    ARBOL[2] = ARBOL[2] + f1Score   # F1 Score
    
    print("/----SIN RE AJUSTE----/")
    print("Score de test: ", scoTest)
    print("Score de train: ", scoTrain)
    print("Precision: ", pre)
    print("Recall:    ", rec)
    print("F1score:   ",f1Score)
    '''
    # Re Ajuste del arbol con el número de profundidad máximo
    arbol = DecisionTreeClassifier(max_depth=profundidadArbol)
    
    # Entrenamiento del modelo con Re Ajuste
    arbol.fit(X_train, y_train)
    
     # Calculo de score en el modelo de test con Re Ajuste
    scoTest = arbol.score(X_test, y_test)
    
    # Calculo de score en el modelo de entrenamiento con Re Ajuste
    scoTrain = arbol.score(X_train, y_train)
    
    # Metricas
    pre = precision_score(y_test,pred)
    rec = recall_score(y_test,pred)
    f1Score = f1_score(y_test,pred)
    
    
    print("/-----CON RE AJUSTE-----/")
    print("Score de test: ", scoTest)
    print("Score de train: ", scoTrain)
    print("Precision: ", pre)
    print("Recall:    ", rec)
    print("F1score:   ",f1Score)'''

In [30]:
#Punto 2
# Estimación de parámetros
# En este código se ilustra el uso de la función que trae la librería sklearn para estimar paramentros,
# la cual no necesita la extracción explícita del conjunto de validación pues hace la estimación usando
# validación cruzada sobre el conjunto de entrenamiento.

from sklearn.svm import SVC
#from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

def RegresionLogistica(sizeTest,  X_train, X_test, y_train, y_test):
    # Se escalan todos los datos
    escalar = StandardScaler()
    X_train = escalar.fit_transform(X_train)
    X_test = escalar.transform(X_test)
    
    # Entrenamiento de los modelos
    algoritmo = LogisticRegression()
    
    algoritmo.fit(X_train,y_train)
    
    # Se realiza una predicción
    y_pred = algoritmo.predict(X_test)
    
    # Matriz de confusión 
    mat = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión")
    print(mat)
    
    # Metricas
    pre = precision_score(y_test,y_pred)
    rec = recall_score(y_test,y_pred)
    f1Score = f1_score(y_test,y_pred)
    
    # Calculo en el vector de las metricas acumulando
    REGRESION[0] = REGRESION[0] + pre       # Presición
    REGRESION[1] = REGRESION[1] + rec       # Recall
    REGRESION[2] = REGRESION[2] + f1Score   # F1 Score
    
    print("Precision: ", pre)
    print("Recall:    ", rec)
    print("F1score:   ",f1Score)
    print("Reporte",classification_report(y_test, y_pred))

In [36]:
def main(N, sizeTest):
    X_train, X_test, y_train, y_test = train_test_split(dataBal.drop(['C'],axis=1), dataBal['C']
                                                        , test_size=sizeTest)
    for i in range(N): # Llamado del agente para revisar las metricas
        print("/-------------------------------------/")
        print("Iteracion: ", i + 1)
        print("SVM")
        print("...")
        SVM_FUN(sizeTest, X_train, X_test, y_train, y_test)
        print("ARBOL DE DECISIÓN")
        print("...")
        ArbolDesicion(sizeTest, 3, X_train, X_test, y_train, y_test)
        print("REGRESIÓN LOGISTICA")
        print("...")
        RegresionLogistica(sizeTest,  X_train, X_test, y_train, y_test)
        
    # Promedios de las metricas SVM
    SVM[0] = SVM[0] / N     # Presición
    SVM[1] = SVM[1] / N     # Recall
    SVM[2] = SVM[2] / N     # F1 Score
    
    # Promedios de las metricas Arboles de decisión
    ARBOL[0] = ARBOL[0] / N     # Presición
    ARBOL[1] = ARBOL[1] / N     # Recall
    ARBOL[2] = ARBOL[2] / N     # F1 Score
    
    # Promedios de las metricas Regresión Logistica
    REGRESION[0] = REGRESION[0] / N     # Presición
    REGRESION[1] = REGRESION[1] / N     # Recall
    REGRESION[2] = REGRESION[2] / N     # F1 Score
    
    print("/--------PROMEDIOS----------/")
    print("* Resultados de promedios SVM")
    print("    Promedio Recall después de {0} iteraciones: {1}".format(N,SVM[0]))
    print("    Promedio Presición después de {0} iteraciones: {1}".format(N,SVM[1]))
    print("    Promedio F1 después de {0} iteraciones: {1}".format(N,SVM[2]))
    print("* Resultados de promedios ARBOLES DE DECISIÓN")
    print("    Promedio Recall después de {0} iteraciones: {1}".format(N,ARBOL[0]))
    print("    Promedio Presición después de {0} iteraciones: {1}".format(N,ARBOL[1]))
    print("    Promedio F1 después de {0} iteraciones: {1}".format(N,ARBOL[2]))
    print("* Resultados de promedios REGRESIÓN LOGISTICA")
    print("    Promedio Recall después de {0} iteraciones: {1}".format(N,REGRESION[0]))
    print("    Promedio Presición después de {0} iteraciones: {1}".format(N,REGRESION[1]))
    print("    Promedio F1 después de {0} iteraciones: {1}".format(N,REGRESION[2]))
    
main(5, 0.4)

/-------------------------------------/
Iteracion:  1
SVM
...
Mejores parametros:
{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
[[2201  899]
 [ 993 2177]]
Precision:  0.7077373211963589
Recall:     0.6867507886435331
F1score:    0.697086135126481
Reporte               precision    recall  f1-score   support

           0       0.69      0.71      0.70      3100
           1       0.71      0.69      0.70      3170

    accuracy                           0.70      6270
   macro avg       0.70      0.70      0.70      6270
weighted avg       0.70      0.70      0.70      6270

ARBOL DE DECISIÓN
...
/----SIN RE AJUSTE----/
Score de test:  0.7700159489633174
Score de train:  1.0
Precision:  0.7732447817836812
Recall:     0.7712933753943217
F1score:    0.7722678458622868
REGRESIÓN LOGISTICA
...
Matriz de confusión
[[2398  702]
 [ 765 2405]]
Precision:  0.7740585774058577
Recall:     0.7586750788643533
F1score:    0.7662896288035684
Reporte               precision    recall  f1-score   suppor

<h1> Actividades a realizar a partir de este script básico: </h1>
1.  Implementar el método holdout para obtener unas métricas de desempeño más confiables. Hacer 5 iteraciones  de las etapas de: partición de los datos - entrenamiento - prueba - calculo de metricas. No olvidar liberar la semilla del generador de números aleatorios.
<br></br>
2.  Adicionar dos métodos para poder comparar su desempeño, los métodos a adicionar son: árbol de decisiones y clasificacador por regresión logística.
<br></br>
3.  Mostrar los resultados comparativos gráficamente, incluyendo la visualización de las curvas ROC por cada método y el valor del área bajo la curva.